# Campito Mountain Bristlecone Example

This notebook uses raw ring widths for bristlecone pine (_Pinus longaeva_) at Campito Mountain in California, USA. This is one of the datasets included in `dplR` as well and include in the dplPy distribution.

> Graybill, D. A. and LaMarche, Jr., V. C. (1983) Campito Mountain Data Set. IGBP PAGES/World Data Center for Paleoclimatology Data Contribution Series 1983-CA533.RWL. NOAA/NCDC Paleoclimatology Program, Boulder, Colorado, USA: https://www.ncei.noaa.gov/pub/data/paleo/treering/measurements/northamerica/usa/ca533.rwl

Let's start by importing some standard Python libraries.  We'll bring in Pandas (for working with 2 dimensional data), Matplotlib to help with plotting,  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 

# the following two lines nicely render figures in the notebook 
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# you can omit the line below if you'd like, but I really don't like the default fonts in Python, so I switch to Helvetica
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['figure.figsize'] = (10, 5)


We can update (or install) `dplpy` directly from this notebook if necessary using the 'bang' (`!`) command - you can just skip this code block or comment it out if you're already running the latest version from the Pypi repository.


In [ ]:
# if necessary, install dplpy
# !pip install dplpy

# if necessary upgrade dplpy to the latest version, v0.1.2 as of January 8th, 2024
# !pip install dplpy --upgrade


Let us now import `dplpy` and we'll check the version number and see where the library is on our system.  If everything is working well, the codeblock should complete without error and you should see that you are running v0.1.2 from one of your Python directories (you will see `/envs/` in the path if you are in a virtual environment)

In [ ]:
import dplpy as dpl

# print out the version number of dplPy you are using - should be v0.1.2 
print('Currently using the following version of dplPy:', dpl.__version__)

# You should see the path to your local Python installation
print('Currently using the following directory for dplPy:', dpl.__file__)

Let's read the Campito Mountain raw measurement series into our notebook now.  We will use the `dpl.readers` method to do this.  For the moment, `.readers` automatically identified the file type from the file suffix, looking for `.rwl` for Tucson formatted decadal files, or `.csv`.  In the future, the method will recognize a wider variety of suffixes (e.g. `.raw`), allow you to pass the file type directly to the method, or pull from the ITRDB.  

In [ ]:
# Read in the Campito Mountain series and assign to a variable
ca533 = dpl.readers("ca533.rwl")

In [ ]:
# take a look at the DataFrame
ca533 #1358 years by 34 columns

We could also read in the Sheep Mountain update (`ca667.rwl`).  This file has headers, though - we tell `.readers` about the existence of the Tucson style (3 line) headers when we call `.readers`:

In [ ]:
# Can specify whether file has headers so reading of data is done appropriately
ca667 = dpl.readers("ca667.rwl", header=True)
ca667 # 310 columns, 4655 years

In [ ]:
print(ca533.index) # Campito Mountain chronology covers 626 to 1983
print(ca667.index)  # Campito Mountain chronology covers -2649 (BCE) to 2005!

We could, if we wanted, use Pandas to merge the two datasets to form a combined Sheep-Campito Mountain chronology.  The codeblock below shows how to use Pandas [.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) to do this.  Because this creates a very large and very long set of series, we won't use this in the rest of the notebook, but it is useful to see how you might do this.  Of note is that the `.merge` function is NOT unique to `dplpy` - following the overall philosophy also adopted for dplR, we don't create functions with openDendro to do common data manipulations or data handling (e.g. things that are not specific to dendrochronology): 

In [ ]:
shpcmp = pd.merge(ca533, ca667, left_index=True, right_index=True, how='outer') # this is just a pure Pandas command
shpcmp # pretty-print from Pandas will show 4655 years x 344 columns

Back to Campito Mountain (`ca533`)now.  Our `dplpy` provides both `.summary` statistics (the individual series mean, quantiles, max and min values) as well as the `.statistics` of the individual series including first and last year, mean, median, standard deviation, skew, gini coefficient and AR1 coefficient similar to ARSTAN.  The module names follow those in `dplR` and report complementary data:

In [ ]:
dpl.summary(ca533)

In [ ]:
dpl.stats(ca533)

Another module available is the `.report`, which has information similar to that provided by COFECHA and mirrors the similar function in dplR, including the years with locally absent rings in each individual series:

In [ ]:
# We can get a report of some of the essential features of this dataset from .report
dpl.report(ca533)

As in dplR, we can readily `.plot` spaghetti (`type="spag"`) or segment (`type="seg"`) plots.  Note that `.plot` here is a method attached to the core `dplpy` library.  

In [ ]:
dpl.plot(ca533, type="seg")
dpl.plot(ca533, type="spag")

Of course, detrending is one of the most important functions of DPL.  

In [ ]:
ca533_rwi = dpl.detrend(ca533, fit="spline", period=200, method="residual", plot=True)

In [ ]:
cam161_spline67 = dpl.detrend(ca533["CAM161"], fit="spline", period=-67,method="residual", plot=True)
cam161_negex = dpl.detrend(ca533["CAM161"], fit="Hugershoff", method="difference", plot=True)

In [ ]:
ca533_crn_std = dpl.chron(ca533_rwi, biweight=True, prewhiten=False, plot=True)
ca533_crn_res = dpl.chron(ca533_rwi, biweight=True, prewhiten=True, plot=True)

## Crossdating Functions

In [ ]:
dpl.xdate(ca533_rwi, prewhiten=True, corr="Spearman", slide_period=50, bin_floor=10, p_val=0.05, show_flags=True)

In [ ]:
dpl.series_corr(ca533_rwi, "CAM181", prewhiten=True, corr="Spearman", seg_length=50, bin_floor=10, p_val=0.05)